In [110]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from scipy.stats import linregress
import datetime
from sqlalchemy import create_engine
import psycopg2

In [111]:
#Reading in Elections CSV files
election_2016_path = "data/countypres_2000-2016.csv"
election_2016_df = pd.read_csv(election_2016_path)

election_2020_path= "data/pres20results.csv"
election_2020_df = pd.read_csv(election_2020_path)

election_2020_total_votes= "data/president_county20.csv"
election_2020_total_votes_df = pd.read_csv(election_2020_total_votes)

In [112]:
#Adding a year column to the election 2020 dataframe
election_2020_df['year']=np.nan

#Filling in the year as 2020 in the 2020 dataframe
election_2020_df['year']=election_2020_df.year.fillna(2020)


In [113]:
#eliminating unecessary columns for election 2016 dataframe
election_2016_df = election_2016_df.drop(columns=['state_po','FIPS','office','version'])

In [114]:
election_2020_total_votes_df = election_2020_total_votes_df.rename(columns={'current_votes': 'totalvotes'})

In [115]:
#cleaning data from the 2020 total votes dataframe
election_2020_total_votes_df = election_2020_total_votes_df.drop(columns=['total_votes','percent'])

In [116]:
#mergring total votes into the 2020 candidate results information
election_2020_merged_df = pd.merge(election_2020_df, election_2020_total_votes_df, how='outer')

In [117]:
#cleaning data and getting rid of win column
election_2020_merged_df = election_2020_merged_df.drop(columns=['won'])

election_2020_merged_df = election_2020_merged_df.rename(columns={'total_votes' : 'candidatevotes'})

In [118]:
#confirming the total counties in each voting year match
county_count_2016 = len(election_2016_df["county"].unique())
county_count_2020 = len(election_2016_df["county"].unique())
print(county_count_2016)
print(county_count_2020)

1854
1854


In [119]:
#merge the 2 dataframes with an outer join
election_merged_df = pd.merge(election_2016_df, election_2020_merged_df, how='outer')

In [120]:
#getting rid of all presidential data prior to 2016
election_merged_df.drop(election_merged_df[election_merged_df['year'] < 2016].index, inplace = True) 

In [121]:
election_2020_merged_df.reset_index(drop=True, inplace=True)

In [122]:
#converting democrat and republican from 2016 files to match 2020 data
election_merged_df = election_merged_df.replace({'democrat':"DEM",'republican':"REP"})

In [123]:
#dropping all write in candidates from 2020 data
election_merged_df.drop(election_merged_df[election_merged_df['party'] == "WRI"].index, inplace = True) 

In [124]:
#dropping all candidates that had 0 votes in a county 
election_merged_df.drop(election_merged_df[election_merged_df['candidatevotes'] == 0].index, inplace = True)

In [125]:
remove = ['LIB', 'GRN', 'WRI', 'PSL', 'IND', 'ALI', 'CST',
       'ASP', 'OTH', 'UTY', 'LLC', 'SWP', 'BAR', 'PRO', 'NON', 'PRG',
       'UNA', 'BMP', 'GOP', 'BFP', 'APV', 'IAP', 'LLP', 'SEP']
election_merged_df = election_merged_df[~election_merged_df['party'].isin(remove)]

In [126]:
election_merged_df = election_merged_df[~election_merged_df.candidate.str.contains("Other")]

In [127]:
#drop the index column
election_merged_df.drop(election_merged_df[election_merged_df['year'] < 2016].index, inplace = True) 

In [128]:
#resetting the index column
election_merged_df.reset_index()

,index,year,state,county,candidate,party,candidatevotes,totalvotes
0,41050,2016,Alabama,Autauga,Hillary Clinton,DEM,5936.0,24973
1,41051,2016,Alabama,Autauga,Donald Trump,REP,18172.0,24973
2,41053,2016,Alabama,Baldwin,Hillary Clinton,DEM,18458.0,95215
3,41054,2016,Alabama,Baldwin,Donald Trump,REP,72883.0,95215
4,41056,2016,Alabama,Barbour,Hillary Clinton,DEM,4871.0,10469
...,...,...,...,...,...,...,...,...
15296,81664,2020,Arizona,La Paz County,Joe Biden,DEM,2236.0,7458
15297,81666,2020,Arizona,Maricopa County,Joe Biden,DEM,1040774.0,2068144
15298,81667,2020,Arizona,Maricopa County,Donald Trump,REP,995665.0,2068144
15299,81669,2020,Arizona,Mohave County,Donald Trump,REP,78535.0,104668


In [129]:
#creating the ID column for the df
election_merged_df.insert(0, 'ID', range(1, 1 + len(election_merged_df)))

In [130]:
#dropping county from the 2020 data 
election_merged_df['county']=election_merged_df['county'].str.replace('County','')

In [131]:
#combining the state and county column
election_merged_df['county_state']=election_merged_df['county'] +","+ election_merged_df['state']

#dropping the county and state columns
election_merged_df= election_merged_df.drop(columns = ['county','state'])

In [132]:
#write the df to a csv file 
election_merged_df.to_csv("election_merged_df.csv", index=False)

In [133]:
#creating the candidate table for postgres
candidates = {'candidate_ID':  ['c1', 'c2','c3'],
        'candidate_name': ['Donald Trump', 'Hilary Clinton','Joe Biden'],
         'party_ID' : ['p1','p2','p2']}

candidates = pd.DataFrame(candidates, columns= ['candidate_ID','candidate_name','party_ID'])

#writing the candidate df to csv
candidates.to_csv("SQL_Data/candidates.csv", index=False)

In [134]:
#replacing candidates with the candidate_IDs in election_merged_df
election_merged_df = election_merged_df.replace({'Donald Trump':"c1",'Hillary Clinton':"c2",'Joe Biden':"c3"})

In [135]:
#creating the parties table for postgres
parties = {'party_ID':  ['p1', 'p2'],
        'party_name': ['republican', 'democrat']}
parties = pd.DataFrame(parties, columns= ['party_ID','party_name'])

#writing the parties df to csv
parties.to_csv("SQL_Data/parties.csv", index=False)

In [136]:
#replacing parties with the party_IDs in election_merged_df
election_merged_df = election_merged_df.replace({'REP':"p1",'DEM':"p2"})

In [137]:
#Creating county_state dataframe for import to postgres
county_state_df = election_merged_df.drop(columns = ['ID','year','candidate','party','candidatevotes','totalvotes'])
county_state_df = county_state_df.drop_duplicates()
county_state_df.rename(columns={'county_state' : 'county_state_ID'})
county_state_df.insert(0, 'county_state_ID', range(1, 1 + len(county_state_df)))

#writing the county_state df to csv
county_state_df.to_csv("SQL_Data/county_state.csv", index=False)

In [138]:
#making a renaming dictionary to replace values in election_merged_df
rename_dict = county_state_df.set_index('county_state').to_dict()['county_state_ID']
election_merged_df = election_merged_df.replace(rename_dict)

In [139]:
#making county_votes_df to pass into CSV for postgres
county_votes_df = election_merged_df.drop(columns = ['ID','party'])
county_votes_df = county_votes_df.rename(columns={'county_state' : 'county_state_ID','candidate':'candidate_ID','totalvotes':'total_votes','candidatevotes':'candidate_votes'})

#writing the parties df to csv
county_votes_df.to_csv("SQL_Data/county_votes.csv", index=False)